In [ ]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
from pylab import cm
import mpltex
import itertools
from matplotlib import pyplot
from scipy import stats

plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42
mpl.rcParams['font.family'] = 'Avenir'
plt.rcParams['font.size'] = 16
plt.rcParams['axes.linewidth'] = 2

In [ ]:
# policies = ["No Investigation", "MinFP", "Random",  "Bayes", "MaxEntropy","All"]

file = open("result_0.3_[1].pkl",'rb')
object_file = pickle.load(file)
file.close()


In [ ]:
temp = {}
for k,v in object_file.items(): 
    temp[k] ={}
    for k2,v2 in v.items():
        if k2.split('_')[0] == "LowFP":
            temp[k]['MinFP_'+ k2.split('_')[1]] = v2
        elif k2.split('_')[0] == "No investigation":
            temp[k]['No Investigation_'+ k2.split('_')[1]] = v2
        else:
            temp[k][k2] = v2
object_file = temp

In [ ]:
save_dict = {}
policies_list = []
metrics_list = []
omega_list = []
for metric, v in object_file.items():
    for k2, v2 in v.items():
        policy = k2.split('_')[0] 
        omega = k2.split('_')[1]
        omega_list.append(omega)
        save_dict[omega] = {}

for metric, v in object_file.items():
    metrics_list.append(metric)
    for k,v in save_dict.items():
        save_dict[k][metric] = {}

for metric, v in object_file.items():
    for k2, v2 in v.items():
        policy = k2.split('_')[0] 
        if policy not in policies_list:
            policies_list.append(policy)
        omega = k2.split('_')[1]
        try:
            save_dict[omega][metric][policy] = v2
        except:
            save_dict[omega][metric] = {}
            save_dict[omega][metric][policy] = v2
policies_list = np.unique(policies_list)
omega_list = list(set(omega_list))


In [ ]:
policies = sorted(policies_list)

In [ ]:
policies

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols=2, figsize=(20,5))

i = 0
locs = ["upper left", "upper right"]
for metric, metric_dict in save_dict[omega_list[-1]].items():
    linestyles = mpltex.linestyle_generator(colors=[],
                                        lines=['-',':'],
                                        markers=['o','^', 'x', 's'],
                                        hollow_styles=[False, True],)
    for policy in metric_dict:
        rollouts = save_dict[omega_list[-1]][metric][policy]
        ax[i].plot(rollouts.mean(axis=0), 
                    lw=2,
                    label=f'{policy}', 
                    **next(linestyles),
                    markevery=5
                    )
        ax[i].set_title('')
    ax[i].legend(loc=locs[i])
    ax[i].set_xlabel('Step (t)', labelpad=10)
    if i == 0:
        ax[i].set_ylabel('MSE', labelpad=10)
    else:
        ax[i].set_ylabel('Entropy', labelpad=10)

    i += 1

fig.savefig('time_vs_mse.eps', format='eps', bbox_inches='tight')
plt.show()


In [ ]:
policy_dict = {}
for policy_omega, rollouts in object_file['MSE'].items():
    try:
        policy_dict[policy_omega.split('_')[0]][policy_omega] = rollouts
    except:
        policy_dict[policy_omega.split('_')[0]] = {}
        policy_dict[policy_omega.split('_')[0]][policy_omega] = rollouts

In [ ]:
diff_omega_dict ={}
for metric in metrics_list:
    diff_omega_dict[metric] = {}
    for policy in policies_list:
        diff_omega_dict[metric][policy] = []
mistakes = []
for omega in omega_list:
    try:
        mistakes.append(1 - float(omega))
    except:
        pass
mistakes = np.sort(mistakes)
for omega, metrics_dict in save_dict.items():
    try:
        for metric, policy_dict in metrics_dict.items():
            for policy, value in policy_dict.items():
                diff_omega_dict[metric][policy].append(np.concatenate([np.array([1 - float(omega)]), value[:,49]], axis = 0))#.mean())
    except:
        pass

for metric in diff_omega_dict.keys():
    for policy in diff_omega_dict[metric].keys():
        diff_omega_dict[metric][policy] = np.array(diff_omega_dict[metric][policy])


In [ ]:
diff_omega_dict['MSE']['Bayes'].shape

In [ ]:
for metric, metric_dict in diff_omega_dict.items():
    for policy, policy_np in metric_dict.items():
        policy_np = np.array(policy_np)
        diff_omega_dict[metric][policy] = policy_np[policy_np[:, 0].argsort()]


In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols=2, figsize=(20,5))

i = 0

locs = ["upper left", "upper right"]
for metric, metric_dict in diff_omega_dict.items():
    for policy, val in metric_dict.items():
        print(policy, val[:,1:].mean(axis=1))
        ax[i].plot(val[:,0], 
                    val[:,1:].mean(axis=1), 
                    lw=2,
                    label=f'{policy}', 
                    **next(linestyles),
                    markevery=1
                    )
        ax[i].set_title('')
        ax[i].set_xscale('log')
        ax[i].legend(loc=locs[i])
        ax[i].set_xlabel('Uncertainty of investigation ($log(1-\omega)$)', labelpad=10)
        ax[i].set_ylabel('MSE', labelpad=10)
    if i == 0:
        ax[i].set_ylabel('MSE', labelpad=10)
    else:
        ax[i].set_ylabel('Entropy', labelpad=10)

    i += 1

fig.savefig('1-omega_vs_entropy.eps', format='eps', bbox_inches='tight')
plt.show()


In [ ]:
omega_list

In [ ]:
omega_list = np.array(omega_list)

In [ ]:
omega = np.array(omega_list, dtype=np.float32)
omega_list = omega_list[omega.argsort()]

In [ ]:
omegas = omega_list[-2:][::-1]


In [ ]:
subset_policy = policies


In [ ]:
omegas

In [ ]:
omega_list

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols=1, figsize=(10,5))

i = 0
locs = ["upper center", "upper right"]
lll = ['-', '--']
mmm = ['o','^', 'x', 's', 'd', '+'][:len(policies)]
ccc = ['black', 'brown']
cc = itertools.cycle(ccc)
ll = itertools.cycle(lll)
mm = itertools.cycle(mmm)

for omega in omegas:
    c = next(cc)
    for policy in sorted(policies):
        if policy != '':
            # print(policy)
            m = next(mm)
            l = next(ll)
            
            rollouts = save_dict[omega]['MSE'][policy]
            print(policy, omega, rollouts.mean(axis = 0)[-1])
            ax.plot(rollouts.mean(axis = 0)[:50], 
                        l,
                        lw=1,
                        label=f'{policy}', 
                        marker= m,
                        color= c,
                        markevery=100
                        )
            ax.set_title('')
    # ax.legend(loc=locs[i], ncol=3)
    ax.set_xlabel('Step (t)', labelpad=10)
    ax.set_ylabel('MSE', labelpad=10)
    ax.set_ylim([0, 0.7])
    
    i += 1

lines = ax.get_lines()
legend1 = pyplot.legend([lines[i] for i in range(len(policies))], policies, ncol = 3, loc="upper center")
ax.add_artist(legend1)
for l in lines:
    l.set_marker('')
legend2 = pyplot.legend([lines[i] for i in [0,len(policies)] ], ["$\omega$=0", "$\omega$=0.05"], loc=(0.06,0.55))
ax.add_artist(legend2)
for i, line in enumerate(lines):
    m = next(mm)
    line.set_marker(m)

for omega in omegas:
    cnt = 0
    c = next(cc)
    for policy in sorted(policies):
        if policy != '':
            print(policy)
            m = next(mm)
            l = next(ll)
            
            rollouts = save_dict[omega]['MSE'][policy]
            
            mean = rollouts.mean(axis = 0)
            std = rollouts.std(axis = 0)
            n= rollouts.shape[0]
            yerr = std / np.sqrt(n) * stats.t.ppf(1-0.05/2, n - 1)

            for i in range(mean.shape[0]):
                
                if (i+cnt)%(10) == 0:
                    bar1 = plt.errorbar(i, 
                                        y=mean[i], 
                                        # linestyle=':', 
                                        yerr=yerr[i],                         
                                        # label=f'{policy}', 
                                        marker=m, 
                                        color=c, 
                                        # markevery=10, 
                                        capsize=4
                                        )
            cnt += 1
fig.savefig('time_vs_mse.eps', format='eps', bbox_inches='tight')
plt.show()


In [ ]:
del diff_omega_dict['Entropy']

In [ ]:
# temp = diff_omega_dict['MSE']['No Investigation'][0,:].copy()
# temp[0] = 0.5

In [ ]:
# for k, np_array in diff_omega_dict['MSE'].items():
#     diff_omega_dict['MSE'][k] = np.concatenate([temp.reshape(1,-1), np_array], axis=0)

In [ ]:
for k, np_array in diff_omega_dict['MSE'].items():
    diff_omega_dict['MSE'][k] = np_array[np.argsort(np_array[:,0])]
    

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols=1, figsize=(10,5))

i = 0

locs = ["upper center", "upper right"]
l = ['-', '--']
m = ['o','^', 'x', 's', 'd', '+']
c = ['black', 'brown']
cc = itertools.cycle(c)
ll = itertools.cycle(l)
mm = itertools.cycle(m)
for metric, metric_dict in diff_omega_dict.items():
    for policy in metric_dict:
        if policy != 'Random':
            m = next(mm)    
            l = next(ll)
            val = metric_dict[policy]
            print(policy, val[:,1:].mean(axis=1))
            ax.plot(val[:,0], 
                    val[:,1:].mean(axis=1), 
                    l,
                    lw=1,
                    label=f'{policy}', 
                    marker = m,
                    color = 'black',
                    markevery=1
                    )
            ax.set_title('')
            ax.set_xscale('log')
            ax.legend(loc=locs[i], ncol=3)
            ax.set_xlabel('The investigation error probability ($log(\omega)$)', labelpad=10)

            

    if i == 0:
        ax.set_ylabel('MSE', labelpad=10)
        ax.set_ylim([0.1, 0.7])
    else:
        ax.set_ylabel('Entropy', labelpad=10)

    i += 1

lines = ax.get_lines()
# legend1 = pyplot.legend([lines[i] for i in [0,1,2,3,4]], subset_policy, ncol = 3, loc="upper center")
# ax.add_artist(legend1)

up_marker = ['_', "1"]
down_marker = ['_', "2"]
up = itertools.cycle(up_marker)
down = itertools.cycle(down_marker)
for metric, metric_dict in diff_omega_dict.items():
    for policy in metric_dict:
        if policy != 'Random':
            u = next(up)    
            d = next(down)
            val = metric_dict[policy]
            mean = val[:,1:].mean(axis=1)
            std = val.std(axis = 1)
            n= val.shape[1]
            yerr = std / np.sqrt(n) * stats.t.ppf(1-0.05/2, n - 1)
            x = val[:,0]
            bar1 = plt.errorbar(x, 
                                y=mean, 
                                linestyle='', 
                                yerr=yerr,                         
                                # label=f'{policy}', 
                                # marker=m, 
                                color='black', 
                                # markevery=10, 
                                capsize=0
                                )
            
            plt.plot(x,mean+yerr, marker=u, ls="", color="black", ms=10)
            plt.plot(x,mean-yerr, marker=d, ls="", color="black", ms=10)
fig.savefig('1-omega_vs_mse.eps', format='eps', bbox_inches='tight')
plt.show()
